# Blink Feature Extraction Tutorial

This tutorial demonstrates how to analyze a long continuous EAR/EOG recording and compute blink features every 30 seconds using the `pyear` package.

We use the sample `ear_eog.fif` file that accompanies the unit tests.

In [3]:
from pathlib import Path

import mne
import pandas as pd
from tqdm import tqdm

from pyblinkers.pipeline import extract_features

## 1. Load the raw recording

In [4]:
fif_path = Path("../unit_test/features/ear_eog_raw.fif")
raw = mne.io.read_raw_fif(fif_path, preload=True)
print(f"Sampling rate: {raw.info["sfreq"]} Hz")

Opening raw data file ..\unit_test\features\ear_eog_raw.fif...
    Reading extended channel information
    Range : 0 ... 179822 =      0.000 ...  1798.220 secs
Ready.
Reading 0 ... 179822  =      0.000 ...  1798.220 secs...
Sampling rate: 100.0 Hz


## 2. Slice the continuous signal into 30-second segments

In [5]:
sfreq = raw.info["sfreq"]
epoch_len = 30.0
end_time = raw.times[-1]
n_epochs = int(end_time // epoch_len)
segments = []
for idx in tqdm(range(n_epochs), desc="Creating segments"):
    start = idx * epoch_len
    stop = start + epoch_len
    segment = raw.copy().crop(tmin=start, tmax=stop, include_tmax=False)
    shifted = mne.Annotations(segment.annotations.onset - start,
                              segment.annotations.duration,
                              segment.annotations.description)
    segment.set_annotations(shifted)
    segments.append(segment)

Creating segments: 100%|██████████| 59/59 [00:00<00:00, 73.51it/s]


## 3. Convert annotations to blink dictionaries

In [6]:
blinks = []
for idx, segment in enumerate(segments):
    signal = segment.get_data(picks="EAR-avg_ear")[0]
    ann = segment.annotations
    for onset, dur, desc in zip(ann.onset, ann.duration, ann.description):
        if desc != "blink":
            continue
        start_frame = int(onset * sfreq)
        end_frame = int((onset + dur) * sfreq)
        blinks.append({
            "refined_start_frame": start_frame,
            "refined_peak_frame": (start_frame + end_frame) // 2,
            "refined_end_frame": end_frame,
            "epoch_signal": signal,
            "epoch_index": idx,
        })

## 4. Compute features for each segment

In [7]:
df = extract_features(blinks, sfreq, epoch_len, n_epochs, raw_segments=segments)
df.head()

,blink_count,blink_rate,ibi_mean,ibi_std,ibi_median,ibi_min,ibi_max,ibi_cv,ibi_rmssd,poincare_sd1,...,blink_half_area_time_std,blink_half_area_time_cv,blink_asymmetry_mean,blink_asymmetry_std,blink_waveform_skewness_mean,blink_waveform_skewness_std,blink_waveform_kurtosis_mean,blink_waveform_kurtosis_std,blink_inflection_count_mean,blink_inflection_count_std
epoch,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The resulting DataFrame contains blink counts, kinematic metrics and other aggregated statistics for each 30-second epoch.